## KG Service
This should be ran inside of the kg-main pod. Please check repo docs to get a Dev Container working.  
Note: Switch host/port/env stuff if you decide to run this elsewhere

In [ ]:
# Sets autoreload of imports to on so that changes in code will immediately show up here.
%load_ext autoreload
%autoreload 2

import requests as r
import pprint as pp
import json
import datetime    
import re
from stores import pg_store
from typing import List, Dict, Literal
from pydantic import BaseModel, Field, validator
import time
from tapipy.tapis import Tapis
from getpass import getpass
import pytz
        
def show(res):
    try:
        pp.pprint(res.json(), width=120) 
    except:
        pp.pprint(res.status_code)
        pp.pprint(res.text, width=120)

In [ ]:
# Create Tapis object and create header_dat
start = time.time()

# Base URL for Tapis
base_url = "https://tacc.develop.tapis.io"
username = "cgarcia"

# Get Tapis object if it isn't already created.
try:
    t
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    print("Creating new Tapis object.")
    try:
        t = Tapis(base_url = base_url,
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

    
# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = None
url = "http://localhost:8000" ## Local

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

## Endpoint Calls

In [ ]:
## Get all pods
res = r.get(f'{url}/pods',
            headers=header_dat)
show(res)

In [ ]:
## Create a pod
res = r.post('http://localhost:8000/pods',
             data=json.dumps({"pod_name": "pod_name_test", "description": "Just a test!"}),
             headers=header_dat)
show(res)

In [ ]:
## Get particular pod
res = r.get(f'{url}/pods/pod_name_test',
            headers=header_dat)
show(res)

## Mess with the models!

In [ ]:
from models import Pod

In [ ]:
## Create a local Pod.
mypod = Pod(pod_name='mypod')
mypod

In [ ]:
## Commit a pod to database
mypod.db_create()

In [ ]:
getmypod = Pod.db_get_with_pk("mypod", tenant='tacc',site='tacc')
getmypod

## Some kubernetes utils. Start a configured neo4j pod.

In [ ]:
from spawner import start_neo4j_pod
from kubernetes_utils import get_current_k8_pods

In [ ]:
start_neo4j_pod("pod_name_here!", revision=1)

In [ ]:
get_current_k8_pods()

## Direct K8 client access.

In [ ]:
from kubernetes import client, config
config.load_incluster_config()
k8 = client.CoreV1Api()

In [ ]:
k8.read_namespaced_pod_status("hello-node-7884b68c89-9fz6h", 'default')

In [ ]:
k8.delete_namespaced_service("servicename", 'default')

## Direct pg_store access.

In [ ]:
from stores import pg_store
engine = pg_store['tacc']['tacc'].engine

In [ ]:
with Session(engine) as session:
    res = session.execute('SELECT * FROM pods')

In [ ]:
with Session(engine) as session:
    j = session.execute('CREATE SCHEMA IF NOT EXISTS "icicle"')

In [ ]:
pg_store['tacc']['tacc'].run("CREATE DATABASE uh", autocommit=True)